### Utils

In [1]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt

from scipy import stats
from scipy import spatial
from scipy.spatial.distance import pdist, cdist, squareform

from sklearn.neighbors import KNeighborsClassifier, NearestCentroid, NearestNeighbors
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score

In [2]:
def compute_distance_list(X, eval_distance_metric='cosine'):
    return spatial.distance.pdist(X, eval_distance_metric)

In [3]:
def string_to_int_mapping(strings):
    # Create a dictionary to store string to integer mapping
    string_to_int = {}
    int_sequence = []

    # Assign a unique integer to each unique string
    for string in strings:
        if string not in string_to_int:
            string_to_int[string] = len(string_to_int)  # Assign the next available integer
        # Append the corresponding integer to the sequence
        int_sequence.append(string_to_int[string])

    return int_sequence, string_to_int

In [4]:
def euclidean_distance_matrix(X_scatter):
    # Compute pairwise Euclidean distances
    pairwise_distances = pdist(X_scatter, metric='euclidean')
    
    # Convert to square matrix form
    distance_matrix = squareform(pairwise_distances)
    
    return distance_matrix

In [5]:
def count_common_elements(list1, list2):
    count_list1 = {}
    count_list2 = {}
    
    for element in list1:
        count_list1[element] = count_list1.get(element, 0) + 1
    
    for element in list2:
        count_list2[element] = count_list2.get(element, 0) + 1
        
    shared_elements_count = 0
    
    for element, count in count_list1.items():
        shared_elements_count += min(count, count_list2.get(element, 0))
    return shared_elements_count

In [6]:
def knn(X_scatter, k=7, distance_function="euclidean"):
    """
    Compute the k-nearest neighbors of the points
    If the distance function is euclidean, the computation relies on faiss-cpu.
    Otherwise, the computation is done based on scikit-learn KD Tree algorithm
    You can use any distance function supported by scikit-learn KD Tree or specify a callable function
    INPUT:
        ndarray: points: list of points
		int: k: number of nearest neighbors to compute
		str or callable: distance_function: distance function to use
    OUTPUT:
		ndarray: knn_indices: k-nearest neighbors of each point 
	"""
    
	
	## make c-contiguous
    points = np.ascontiguousarray(X_scatter, dtype=np.float32)

    if distance_function == "euclidean":
        index = faiss.IndexFlatL2(points.shape[1])
        index.add(points)
        knn_indices = index.search(points, k+1)[1][:, 1:]
	
    return knn_indices

In [7]:
def knn_with_ranking(points, k, distance_matrix):
  knn_indices = np.empty((points.shape[0], k), dtype=np.int32)
  ranking = np.empty((points.shape[0], points.shape[0]), dtype=np.int32)
  
  for i in range(points.shape[0]):
    distance_to_i = distance_matrix[i]
    sorted_indices = np.argsort(distance_to_i)
    knn_indices[i] = sorted_indices[1:k+1]
    ranking[i] = np.argsort(sorted_indices)
  
  return knn_indices, ranking

### Global Measures

Our analysis of the global similarity between two scatter plots relies on the following four metrics <br>
1. Pearson's correlation coefficient
2. Spearman's rank correlation coefficient
3. Cluster Ordering, and the
4. Rotation derived from Procrustes analysis

We ignore metrics that are not normalized, e.g., Stress, Kullback-Leiber Divergence, Distance-to-Measure, or the Topographic Product.

In [8]:
# Spearman's correlation coefficient
def metric_spearman_correlation(D_scatter1, D_scatter2):
    return stats.spearmanr(D_scatter1, D_scatter2)[0]

In [9]:
# Pearson's rank correlation coefficient
def metric_pearson_correlation(D_scatter1, D_scatter2):
    return stats.pearsonr(D_scatter1, D_scatter2)[0]

In [10]:
# Cluster Ordering
def metric_cluster_ordering(X_scatter1, X_scatter2, y):
    #clf_scatter1 = NearestCentroid().fit(X = df_scatter1, y = y)
    #clf_scatter2 = NearestCentroid().fit(X = df_scatter2, y = y)
    clf_scatter1 = NearestCentroid().fit(X = X_scatter1, y = y)
    clf_scatter2 = NearestCentroid().fit(X = X_scatter2, y = y)
    
    D_scatter1 = compute_distance_list(clf_scatter1.centroids_, 'euclidean')
    D_scatter2 = compute_distance_list(clf_scatter2.centroids_, 'euclidean')
    
    return metric_spearman_correlation(D_scatter1, D_scatter2)

In [11]:
metric_cluster_ordering(X_scatter1, X_scatter2, Y)

NameError: name 'X_scatter1' is not defined

In [ ]:
# Rotation derived from Procrustes analysis
def center(X_scatter):
    n,m = X_scatter.shape
    x_mean = sum(X_scatter[:,0].tolist())/n
    y_mean = sum(X_scatter[:,1].tolist())/n
    return X_scatter - [x_mean, y_mean]


# https://stackoverflow.com/questions/18925181/procrustes-analysis-with-numpy
def procrustes_manual(X, Y, scaling=True, reflection='best'):
    """
    A port of MATLAB's `procrustes` function to Numpy.

    Procrustes analysis determines a linear transformation (translation,
    reflection, orthogonal rotation and scaling) of the points in Y to best
    conform them to the points in matrix X, using the sum of squared errors
    as the goodness of fit criterion.

        d, Z, [tform] = procrustes(X, Y)

    Inputs:
    ------------
    X, Y    
        matrices of target and input coordinates. they must have equal
        numbers of  points (rows), but Y may have fewer dimensions
        (columns) than X.

    scaling 
        if False, the scaling component of the transformation is forced
        to 1

    reflection
        if 'best' (default), the transformation solution may or may not
        include a reflection component, depending on which fits the data
        best. setting reflection to True or False forces a solution with
        reflection or no reflection respectively.

    Outputs
    ------------
    d       
        the residual sum of squared errors, normalized according to a
        measure of the scale of X, ((X - X.mean(0))**2).sum()

    Z
        the matrix of transformed Y-values

    tform   
        a dict specifying the rotation, translation and scaling that
        maps X --> Y

    """

    n,m = X.shape
    ny,my = Y.shape

    muX = X.mean(0)
    muY = Y.mean(0)

    X0 = X - muX
    Y0 = Y - muY

    ssX = (X0**2.).sum()
    ssY = (Y0**2.).sum()

    # centred Frobenius norm
    normX = np.sqrt(ssX)
    normY = np.sqrt(ssY)

    # scale to equal (unit) norm
    X0 /= normX
    Y0 /= normY

    if my < m:
        Y0 = np.concatenate((Y0, np.zeros(n, m-my)),0)

    # optimum rotation matrix of Y
    A = np.dot(X0.T, Y0)
    U,s,Vt = np.linalg.svd(A,full_matrices=False)
    V = Vt.T
    T = np.dot(V, U.T)

    if reflection != 'best':

        # does the current solution use a reflection?
        have_reflection = np.linalg.det(T) < 0

        # if that's not what was specified, force another reflection
        if reflection != have_reflection:
            V[:,-1] *= -1
            s[-1] *= -1
            T = np.dot(V, U.T)

    traceTA = s.sum()

    if scaling:

        # optimum scaling of Y
        b = traceTA * normX / normY

        # standarised distance between X and b*Y*T + c
        d = 1 - traceTA**2

        # transformed coords
        Z = normX*traceTA*np.dot(Y0, T) + muX

    else:
        b = 1
        d = 1 + ssY/ssX - 2 * traceTA * normY / normX
        Z = normY*np.dot(Y0, T) + muX

    # transformation matrix
    if my < m:
        T = T[:my,:]
    c = muX - b*np.dot(muY, T)
    
    #transformation values 
    tform = {'rotation':T, 'scale':b, 'translation':c}
   
    return d, Z, tform

def get_rotation_angle(rotation_matrix):
    """
    Calculate the rotation angle from a 2x2 rotation matrix.
    """
    angle_rad = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
    angle_deg = np.degrees(angle_rad)
    return angle_deg

### Local Metrics

In [12]:
# Trustworthiness
def metric_trustworthiness(X_scatter1, X_scatter2, D_scatter1_square, D_scatter2_square, k=7):
    #D_scatter1 = spatial.distance.squareform(D_scatter1)
    #D_scatter2 = spatial.distance.squareform(D_scatter2)
    
    n = X_scatter1.shape[0]
    
    nn_scatter1 = D_scatter1_square.argsort()
    nn_scatter2 = D_scatter2_square.argsort()
    
    knn_scatter1 = nn_scatter1[:, :k + 1][:, 1:]
    knn_scatter2 = nn_scatter2[:, :k + 1][:, 1:]
    
    sum_i = 0
    
    for i in range(n):
        U = np.setdiff1d(knn_scatter2[i], knn_scatter1[i])
        
        sum_j = 0
        for j in range(U.shape[0]):
            sum_j += np.where(nn_scatter1[i] == U[j])[0] - k
        
        sum_i += sum_j
        
    return float((1 - (2 / (n * k * (2 * n - 3 * k -1)) * sum_i)).squeeze())

In [13]:
# Continuity
def metric_continuity(X_scatter1, X_scatter2, D_scatter1_square, D_scatter2_square, k=7):
    n = X_scatter1.shape[0]

    nn_scatter1 = D_scatter1_square.argsort()
    nn_scatter2 = D_scatter2_square.argsort()

    knn_scatter1 = nn_scatter1[:, :k + 1][:, 1:]
    knn_scatter2 = nn_scatter2[:, :k + 1][:, 1:]

    sum_i = 0

    for i in range(n):
        V = np.setdiff1d(knn_scatter1[i], knn_scatter2[i])

        sum_j = 0
        for j in range(V.shape[0]):
            sum_j += np.where(nn_scatter2[i] == V[j])[0] - k

        sum_i += sum_j

    return float((1 - (2 / (n * k * (2 * n - 3 * k - 1)) * sum_i)).squeeze())

In [14]:
# Label Preservation
def metric_label_preservation(X_scatter1, X_scatter2, y, k=7):
    #arr_scatter1 = scatter1.values
    #arr_scatter2 = scatter2.values
    label_preservation = 0
    
    nbrs_scatter1 = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(X_scatter1)
    nbrs_scatter2 = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(X_scatter2)
    
    for i in range(X_scatter1.shape[0]):
        distances_scatter1, indices_scatter1 = nbrs_scatter1.kneighbors(X_scatter1[i].reshape(1, -1))
        distances_scatter2, indices_scatter2 = nbrs_scatter2.kneighbors(X_scatter2[i].reshape(1, -1))
        
        indices_scatter1_list = indices_scatter1[0].tolist()
        indices_scatter2_list = indices_scatter2[0].tolist()
        
        labels_scatter1 = [y[index] for index in indices_scatter1_list]
        labels_scatter2 = [y[index] for index in indices_scatter2_list]
        
        label_preservation_pointwise = (count_common_elements(labels_scatter1, labels_scatter2) - 1)/7
        label_preservation += label_preservation_pointwise
    
    return label_preservation/X_scatter1.shape[0]

In [15]:
# Local Continuity Meta-Criterion (Jeon Hong)
def metric_local_continuity(X_scatter1, X_scatter2, D_scatter1_square, D_scatter2_square, k=7):
    nn_scatter1 = D_scatter1_square.argsort()
    nn_scatter2 = D_scatter2_square.argsort()
    
    knn_scatter1 = nn_scatter1[:, :k + 1][:, 1:]
    knn_scatter2 = nn_scatter2[:, :k + 1][:, 1:]

    point_num = X_scatter1.shape[0]
    local_distortion_list = []

    for i in range(point_num):
        local_distortion_list.append(np.intersect1d(knn_scatter1[i], knn_scatter2[i]).shape[0] -((k*k)/(point_num - 1)))

    local_distortion_list = np.array(local_distortion_list)
    local_distortion_list = local_distortion_list / k 
  
    average_distortion = np.mean(local_distortion_list)
    return average_distortion

In [16]:
def metric_mrre_help(base_ranking,target_ranking, target_knn_indices, k=7):
	local_distortion_list = []
	points_num = target_knn_indices.shape[0]
	for i in range(points_num):
		base_rank_arr   = base_ranking[i][target_knn_indices[i]]
		target_rank_arr = target_ranking[i][target_knn_indices[i]]
		local_distortion_list.append(np.sum(np.abs(base_rank_arr - target_rank_arr) / target_rank_arr))
	
	c = sum([abs(points_num - 2 * i + 1) / i for i in range(1, k + 1)])
	local_distortion_list = np.array(local_distortion_list)
	local_distortion_list = 1 - local_distortion_list / c

	average_distortion = np.mean(local_distortion_list)

	return average_distortion


def metric_mrre(X_scatter1, X_scatter2,D_scatter1_square, D_scatter2_square, k=7):
	scatter1_knn_indices, scatter1_ranking = knn_with_ranking(X_scatter1, k, D_scatter1_square)
	scatter2_knn_indices, scatter2_ranking = knn_with_ranking(X_scatter2, k, D_scatter2_square)
	

	mrre_false = metric_mrre_help(scatter1_ranking, scatter2_ranking, scatter2_knn_indices, k)
	mrre_missing = metric_mrre_help(scatter2_ranking, scatter1_ranking, scatter1_knn_indices, k)

	return {
		"mrre_false": mrre_false,
		"mrre_missing": mrre_missing,
	}

### Cluster-level Measures

In [17]:
# Distance Consistency
def single_metric_distance_consistency(X_scatter, y):
    clf = NearestCentroid()
    clf.fit(X=X_scatter, y=y)
    nearest_centroids = clf.predict(X=X_scatter)
    num_same_label = sum([1 if y[i] == nearest_centroids[i] else 0 for i in range(len(y))])
    return num_same_label / len(y)

def metric_distance_consistency(X_scatter1, X_scatter2, y):
    return abs(single_metric_distance_consistency(X_scatter1, y) - single_metric_distance_consistency(X_scatter2, y))

In [18]:
# Silhouette Coefficient
def single_metric_silhouette_coefficient(X_scatter, y):
    return silhouette_score(X_scatter, y, metric='euclidean')

def metric_silhouette_coefficient(X_scatter1, X_scatter2, y):
    return abs(silhouette_score(X_scatter1, y, metric='euclidean') - silhouette_score(X_scatter2, y, metric='euclidean'))

In [19]:
# Calinski-Harabasz-Score
def single_metric_calinski_harabasz_score(X_scatter, y):
    return calinski_harabasz_score(X_scatter, y)

def metric_calinski_harabasz_score(X_scatter1, X_scatter2, y):
    return abs(calinski_harabasz_score(X_scatter1, y) - calinski_harabasz_score(X_scatter2, y))

In [20]:
# Davies-Boulding-Index
def single_metric_davies_bouldin_score(X_scatter, y):
    return davies_bouldin_score(X_scatter, y)

def metric_davies_bouldin_score(X_scatter1, X_scatter2, y):
    return abs(davies_bouldin_score(X_scatter1, y) - davies_bouldin_score(X_scatter2, y))

### Example

In [2]:
# load first scatter plot
df_scatter1 = pd.read_csv("results/results1_7Categories.csv")

# load second scatter plot
df_scatter2 = pd.read_csv("results/results2_7Categories.csv")
Y,_ = string_to_int_mapping(df_scatter2['labels'].tolist()) # both scatter plots have the same labels

FileNotFoundError: [Errno 2] No such file or directory: 'results/results1_7Categories.csv'

In [ ]:
# Checking Tims Label Preservation
# load first scatter plot
df_scatter1 = pd.read_csv("D:/result/20_newsgroups/umap_20_newsgroups_nmf_tfidf_n_topics_20_5_0.8_cosine_1.0_1.0_1_1.0_5_58aaa17c4ffbd81_6377d5b9c5fe1a5.npy")
# umap_20_newsgroups_nmf_tfidf_n_topics_20_5_0.8_cosine_1.0_1.0_1_1.0_5_58aaa17c4ffbd81_6377d5b9c5fe1a5.npy

# load second scatter plot
df_scatter2 = pd.read_csv("results/results2_7Categories.csv")
Y,_ = string_to_int_mapping(df_scatter2['labels'].tolist()) # both scatter plots have the same labels

In [ ]:
# Create a figure with two subplots arranged horizontally
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# First scatter plot
ax1.scatter(
    x=df_scatter1['x'].tolist(), 
    y=df_scatter1['y'].tolist(), 
    c=Y, 
    cmap=plt.cm.get_cmap('Paired'), 
    alpha=1.0)
ax1.set_title('Scatter Plot 1')

# Second scatter plot
ax2.scatter(
    x=df_scatter2['x'].tolist(), 
    y=df_scatter2['y'].tolist(), 
    c=Y, 
    cmap=plt.cm.get_cmap('Paired'), 
    alpha=1.0)
ax2.set_title('Scatter Plot 2')

# Display the plot
plt.show()

In [ ]:
X_scatter1 = df_scatter1[['x','y']].values
X_scatter2 = df_scatter2[['x','y']].values

D_scatter1 = compute_distance_list(X_scatter1, eval_distance_metric='euclidean')
D_scatter2 = compute_distance_list(X_scatter2, eval_distance_metric='euclidean')

D_scatter1_square = euclidean_distance_matrix(X_scatter1)
D_scatter2_square = euclidean_distance_matrix(X_scatter2)

Y,_ = string_to_int_mapping(df_scatter1['labels'].tolist())

# Global measures
print("GLOBAL MEASURES")
print("Spearman Correlation: " + str(metric_spearman_correlation(D_scatter1, D_scatter2)))
print("Pearson Correlation: " + str(metric_pearson_correlation(D_scatter1, D_scatter2)))
print("Cluster Ordering: " + str(metric_cluster_ordering(X_scatter1, X_scatter2, Y)))
d,Z,tform = procrustes_manual(center(X_scatter1), center(X_scatter2), reflection="False")
print("Rotation: " + str(get_rotation_angle(tform["rotation"])))
print("")
print("CLUSTER-LEVEL MEASURES")
print("Distance Consistency: " + str(metric_distance_consistency(X_scatter1, X_scatter2, Y)))
print("Silhouette Coefficient: " + str(metric_silhouette_coefficient(X_scatter1, X_scatter2, Y)))
# print("Calinski Harabasz Score: " + str(metric_calinski_harabasz_score(X_scatter1, X_scatter2, Y))) # does not have a predefined value range
# print("Davies-Bouldin Score: " + str(metric_davies_bouldin_score(X_scatter1, X_scatter2, Y))) # does not have a predefined value range
print("")
print("LOCAL MEASURES")
print("Trustworthiness: " + str(metric_trustworthiness(X_scatter1, X_scatter2, D_scatter1_square, D_scatter2_square, k=7)))
print("Continuity: " + str(metric_continuity(X_scatter1, X_scatter2, D_scatter1_square, D_scatter2_square, k=7)))
print("Label Preservation: " + str(metric_label_preservation(X_scatter1, X_scatter2, Y, k=7)))
print("Local Continuity: " + str(metric_local_continuity(X_scatter1, X_scatter2, D_scatter1_square, D_scatter2_square, k=7)))
print("MRRE missing: " + str(metric_mrre(X_scatter1, X_scatter2,D_scatter1_square, D_scatter2_square, k=7)['mrre_missing']))
print("MRRE false: " + str(metric_mrre(X_scatter1, X_scatter2,D_scatter1_square, D_scatter2_square, k=7)['mrre_false']))